In [1]:
from model import *
from helpers import *

In [2]:
# Run model over sampled parameter combinations and collect metrics

def run_one(params,
            dt=1e4/1e6,
            verbose=False,
            Cimb_min_guess=0.4,
            max_t_max=100,
            safety_pad=10,
            min_window=10):
    
    # Dynamically guess t_max using tau and W_LIP/C_imb
    tau = float(params['tau'])
    C_imb_effect = float(params['W_LIP'] if 'W_LIP' in params else params['C_imb'])

    if np.isfinite(tau) and tau > 0 and C_imb_effect > 0:
        guess_duration = -tau * np.log(Cimb_min_guess / C_imb_effect)
        t_max = min(max(guess_duration + safety_pad, min_window), max_t_max)
    else:
        t_max = min_window

    if verbose:
        print(f"Running with t_max = {t_max:.0f} Myr")

    # run the model
    try:
        results = run_model(t_max=t_max, dt=dt, **params)

    # catch negative Worg error
    except Exception as e:
        if verbose:
            print(f"run_one failed: {e}. Returning NaNs.")
        return {
            'snowball_num': np.nan,
            'end_time': np.nan,
            'LIP_volume_end': np.nan,
            't_max_used': t_max,
        }
    
    # timing diagnostics
    t = results['t']
    snowball = results['snowball']
    sb_starts, sb_ends, sb_durs, ig_durs = get_times(t, snowball)
    end_time = sb_ends[-1] if len(sb_ends) > 0 else (t[-1] if len(t) > 0 else np.nan)
   
    # LIP diagnostics
    lip_t, lip_vol, lip_height = LIP_volume(results, verbose=False)
    lip_vol_end = lip_vol[-1] if lip_vol is not None and len(lip_vol) > 0 else np.nan
    res = {
        'snowball_num': len(sb_starts),
        'end_time': end_time,
        'LIP_volume_end': lip_vol_end,
        't_max_used': t_max,
    }
    
    if verbose:
        print(f"Result: {len(sb_starts)} snowballs, end_time={end_time:.2f} Myr")
    return res

In [4]:
# make grid to be sampled
import itertools

# uncomment for linear spacing
#Cimb_arr = np.linspace(0.01, 40, 20)
#tau_arr  = np.linspace(0.1, 80, 20)

# log spacing for better resolution at low values
Cmin, Cmax, N_C = 0.1, 40.0, 100
Tmin, Tmax, N_T = 0.1, 80.0, 100

Cimb_arr = np.logspace(np.log10(Cmin), np.log10(Cmax), N_C)
tau_arr  = np.logspace(np.log10(Tmin), np.log10(Tmax), N_T)

param_grid = list(itertools.product(Cimb_arr, tau_arr))
param_dicts = [{'C_imb': Cimb, 'tau': tau} for Cimb, tau in param_grid]

In [5]:
# quick test run
run_one(param_dicts[0], verbose=True)

Running with t_max = 10 Myr
Result: 0 snowballs, end_time=10.01 Myr


{'snowball_num': 0,
 'end_time': np.float64(10.009999999999831),
 'LIP_volume_end': np.float64(0.0),
 't_max_used': 10}

In [6]:
# parallelize running over all samples
from joblib import Parallel, delayed
from tqdm.auto import tqdm

n_workers = -1 #-1 uses all available cores

results_list = Parallel(n_jobs=n_workers, backend='loky')(
    (delayed(run_one)(param_dicts[i]) for i in tqdm(range(len(param_dicts))))
)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [7]:
# combine parameter dicts with results for each run
import pandas as pd

assert len(param_dicts) == len(results_list), (
    f"Mismatch: {len(param_dicts)} params vs {len(results_list)} results")

combined_records = [
    {**param_dicts[i], **results_list[i]}
    for i in range(len(param_dicts))
]

df = pd.DataFrame(combined_records)
df = df.sort_values(by=['tau', 'C_imb'])
df.head()


,C_imb,tau,snowball_num,end_time,LIP_volume_end,t_max_used
0,0.100000,0.1,0,10.01,0.0,10.0
100,0.106239,0.1,0,10.01,0.0,10.0
200,0.112867,0.1,0,10.01,0.0,10.0
300,0.119909,0.1,0,10.01,0.0,10.0
400,0.127390,0.1,0,10.01,0.0,10.0


In [8]:
# save results
csv_path = 'data/grid_for_contour.csv'
df.to_csv(csv_path, index=False)